In [5]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://localhost:5050")
client = MlflowClient()
model_name = "dtc_persona_clustering_model"

# You've already found the latest version, let's say its version number is '19'
latest_version_info = client.get_latest_versions(name=model_name, stages=["None"])[0]
version_to_promote = latest_version_info.version

print(f"Promoting version {version_to_promote} of model '{model_name}' to 'Production'.")

# This is the key step:
client.transition_model_version_stage(
    name=model_name,
    version=version_to_promote,
    stage="Production",
    archive_existing_versions=True # This will move any existing 'Production' model to 'Archived'
)

print("Promotion successful.")

Promoting version 19 of model 'dtc_persona_clustering_model' to 'Production'.
Promotion successful.


/var/folders/2p/knkmky2x69s7jl6b1s6y8s3r0000gn/T/ipykernel_46089/3804978902.py:9: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version_info = client.get_latest_versions(name=model_name, stages=["None"])[0]
/var/folders/2p/knkmky2x69s7jl6b1s6y8s3r0000gn/T/ipykernel_46089/3804978902.py:15: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [6]:
import mlflow


# Set the tracking URI to point to your MLflow server
mlflow.set_tracking_uri("http://localhost:5050")

MODEL_NAME = "dtc_persona_clustering_model"
MODEL_STAGE = "Production"
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_STAGE}"

print(f"Loading model from: {MODEL_URI}")
# MLflow will automatically resolve this to the correct GCS path and download it.
loaded_model = mlflow.pyfunc.load_model(MODEL_URI)
print("Model loaded successfully.")


Loading model from: models:/dtc_persona_clustering_model/Production


2025/07/22 09:47:06 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - lz4 (current: uninstalled, required: lz4==4.4.4)
 - pandas (current: 2.2.3, required: pandas==1.5.3)
 - psutil (current: 7.0.0, required: psutil==5.9.8)
 - scipy (current: 1.13.1, required: scipy==1.15.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/07/22 09:47:06 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.17`, differs from the version of Python that is currently running, `Python 3.9.22`, and may be incompatible


Model loaded successfully.


In [7]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 265157f127dd427a8355e882381df322

### load from registry with exception catch

In [8]:
MLFLOW_TRACKING_URI = "http://localhost:5050"
MODEL_NAME = "dtc_persona_clustering_model"
MODEL_STAGE = "Production"
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_STAGE}"

# Initialize a variable to hold the model
loaded_model = None

try:
    # --- Step 1: Set the tracking URI ---
    # This tells the MLflow client where to send requests.
    print(f"Connecting to MLflow Tracking Server at: {MLFLOW_TRACKING_URI}")
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    # --- Step 2: Load the model ---
    # This is the primary operation that might fail if the server is unreachable
    # or the model/stage does not exist.
    print(f"Loading model '{MODEL_NAME}' from stage '{MODEL_STAGE}'...")
    loaded_model = mlflow.pyfunc.load_model(model_uri=MODEL_URI)

# Catch specific MLflow exceptions for better error messages
except mlflow.exceptions.RestException as e:
    print("\n--- MLFLOW ERROR ---")
    print(f"Failed to load the model due to an MLflow REST API error.")
    print("This commonly happens if:")
    print(f"  1. The MLflow server is not running or accessible at '{MLFLOW_TRACKING_URI}'.")
    print(f"  2. The model named '{MODEL_NAME}' does not exist in the registry.")
    print(f"  3. The model does not have a version assigned to the '{MODEL_STAGE}' stage.")
    print(f"\nOriginal Error: {e}")
    # Exit the script with an error code, as the application cannot continue.
    sys.exit(1)

# Catch any other unexpected exceptions during the process
except Exception as e:
    print(f"\n--- AN UNEXPECTED ERROR OCCURRED ---")
    print(f"An error occurred while trying to load the model: {e}")
    sys.exit(1)

# The 'else' block runs only if the 'try' block completes successfully
else:
    print("\n✅ Model loaded successfully!")
    # You can now use the 'loaded_model' object for predictions.
    # For example, print its metadata if available.
    if hasattr(loaded_model, 'metadata'):
        print(f"Model Signature: {loaded_model.metadata.signature}")

Connecting to MLflow Tracking Server at: http://localhost:5050
Loading model 'dtc_persona_clustering_model' from stage 'Production'...


2025/07/22 09:53:04 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - lz4 (current: uninstalled, required: lz4==4.4.4)
 - pandas (current: 2.2.3, required: pandas==1.5.3)
 - psutil (current: 7.0.0, required: psutil==5.9.8)
 - scipy (current: 1.13.1, required: scipy==1.15.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/07/22 09:53:04 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.17`, differs from the version of Python that is currently running, `Python 3.9.22`, and may be incompatible



✅ Model loaded successfully!
Model Signature: inputs: 
  ['x1': double (required), 'x2': double (required), 'x3': double (required), 'x4': double (required), 'x5': double (required), 'x6': double (required), 'x7': double (required), 'x8': double (required), 'x9': double (required), 'x10': double (required)]
outputs: 
  [Tensor('int32', (-1,))]
params: 
  None



In [9]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 265157f127dd427a8355e882381df322